In [1]:
import tensorflow as tf
import numpy as np
import json
import joblib  # For loading scalers
from sklearn.preprocessing import MinMaxScaler

# Load the trained ANN model
model = tf.keras.models.load_model("route_prediction_ann_v2.h5")

#  Load the fitted scalers (saved during training)
scaler_X = joblib.load("scaler_X.pkl")
scaler_y = joblib.load("scaler_y.pkl")

# Load traffic data from JSON
with open("traffic_data.json") as f:
    traffic_data = json.load(f)

# Function to fetch data from JSON and predict route details
def predict_route(start_loc, dest_loc):
    try:
        route_info = traffic_data[start_loc][dest_loc]
    except KeyError:
        return "No traffic data available for this route."
    
    # Extract JSON data
    distance = route_info["distance"]
    num_signals = route_info["num_signals"]
    vehicle_count = route_info["vehicle_count"]
    red_light_delay = route_info["red_light_delay"]
    current_pollution = route_info["current_pollution"]

    # Compute rush factor and pollution exposure based on the data
    rush_factor_car = vehicle_count / 300  
    rush_factor_bike = rush_factor_car * 0.8
    rush_factor_cycle = 0  
    rush_factor_bus = 0  
    rush_factor_metro = 0.4  
    pollution_exposure_car = current_pollution
    pollution_exposure_bike = pollution_exposure_car * 1.2
    pollution_exposure_cycle = pollution_exposure_car * 1.5
    pollution_exposure_bus = pollution_exposure_car * 0.8
    pollution_exposure_metro = pollution_exposure_car * 0.5

    # Prepare input for ANN
    input_data = np.array([[distance, num_signals, red_light_delay, vehicle_count,
                            rush_factor_car, rush_factor_bike, rush_factor_cycle, rush_factor_bus, rush_factor_metro,
                            pollution_exposure_car, pollution_exposure_bike, pollution_exposure_cycle,
                            pollution_exposure_bus, pollution_exposure_metro]])

    # Normalize input using the pre-fitted scaler
    input_scaled = scaler_X.transform(input_data)
    
    # Get predictions
    prediction_scaled = model.predict(input_scaled)
    
    #  Reverse normalization to get actual values
    output = scaler_y.inverse_transform(prediction_scaled)
    
    return {
        "Time_Car": output[0][0],
        "Time_Bike": output[0][1],
        "Time_Cycle": output[0][2],
        "Time_Bus": output[0][3],
        "Time_Metro": output[0][4],
        "Expense_Car": output[0][5],
        "Expense_Bike": output[0][6],
        "Expense_Cycle": output[0][7],
        "Expense_Bus": output[0][8],
        "Expense_Metro": output[0][9],
        "Pollution_Car": output[0][10],
        "Pollution_Bike": output[0][11],
        "Pollution_Cycle": output[0][12],
        "Pollution_Bus": output[0][13],
        "Pollution_Metro": output[0][14]
    }

# Example usage
start_location = "Connaught Place"
destination = "Dwarka"
prediction = predict_route(start_location, destination)

print("Predicted Results:")
for key, value in prediction.items():
    print(f"{key}: {value:.2f}")


C:\Users\adars\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
Predicted Results:
Time_Car: 45.84
Time_Bike: 27.38
Time_Cycle: 71.53
Time_Bus: 42.55
Time_Metro: 34.47
Expense_Car: 95.19
Expense_Bike: 48.91
Expense_Cycle: 0.03
Expense_Bus: 25.31
Expense_Metro: 31.68
Pollution_Car: 195.38
Pollution_Bike: 113.44
Pollution_Cycle: -0.11
Pollution_Bus: 108.15
Pollution_Metro: 28.13
